--> **amazone satelite data** (output : Deforestation and Pollution Probability ) : <a href="https://drive.google.com/file/d/1AAlKyamVlFkhesB-FVYRyw9Y_XIX0hj8/view?usp=sharing" target="_blank">drive/MyDrive/latest_model.h5</a>

--> **satelite image classification** ( output_labels : cloudy,water,green_area,desert) :<a href="https://drive.google.com/file/d/1--aPWWVaDvJ7ilaRFP_cPHRJXty3ebz2/view?usp=sharing" target="_blank">drive/MyDrive/latest_satelite_image.keras</a>

--> **Iot Data Air Quality Prediction** (output_labels:Good,Moderate,Unhealthy_for_Sensitive_Groups,Unhealthy,Very_Unhealthy,Severe): Pipeline
<a href="https://drive.google.com/file/d/1-JL8llYQ_H_7dUWM4jRzGyUzseyevYvK/view?usp=sharing" target="_blank">drive/MyDrive/latest_iot.pkl</a>

--> **Iot Data Air Quality Prediction** (output_labels:Good,Moderate,Unhealthy_for_Sensitive_Groups,Unhealthy,Very_Unhealthy,Severe): Model
<a href="https://drive.google.com/file/d/1-IYHwciNBazU1kH_tb7A7I0Drm06OZjc/view?usp=sharing" target="_blank">drive/MyDrive/latest_iot.h5</a>

****************************************************************************************

##### IMPORTANT LIBRARIES FOR LOAD MODEL 
###### FOR KERAS MODEL
` from tensorflow.keras.models import load_model `


`from tensorflow.keras.preprocessing import image`<br/>

`import numpy as np`<br/>

`from tensorflow.keras.losses import SparseCategoricalCrossentropy`<br/>

`import keras`<br/>

`import tensorflow as tf`<br/>

###### FOR PKL MODELS
` import pickle as pkl `
<hr/>

### IMPORTANT FUNCTION FOR LOADING SATELITE_IMAGE MODEL

In [ ]:
def custom_loss_from_config(config):
    if config['class_name'] == 'SparseCategoricalCrossentropy':
        config['config'].pop('fn', None)  # Remove the 'fn' key if it exists
        return SparseCategoricalCrossentropy.from_config(config['config'])
    return keras.losses.get(config)


<hr>

##### EXAMPLES
` model1=load_model("latest_model.h5") `<br>
` model2=load_model("latest_satelite_image.h5",custom_objects={'SparseCategoricalCrossentropy': custom_loss_from_config})`<br>
` model3_pipe=pkl.load(open("latest_iot.pkl","rb"))`<br>
` model3_keras=load_model("latest_iot.h5")`<br>

<hr>

##### FUNCTIONS FOR MODELS

#### FOR DEFORESTATION

  

In [ ]:
from keras.preprocessing import image  
def predict_deforestation_pollution(model, img_path, img_size=(224, 224)):

    img = image.load_img(img_path, target_size=img_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    preds = model.predict(x)

    return preds

<hr>

#####  FOR OUTPUT LABELS LIKE CLOUDY,GREEN_AREA,DESERT,WATER

In [ ]:
def satelite_image_classification(model,img_path,img_size=(72,128)):
      classes = {
      0: "cloudy",
      1: "desert",
      2: "water",
      3: "green_area"
      }
      sample_image_path=img_path
      sample_image = image.load_img(sample_image_path, target_size=img_size)
      sample_image = image.img_to_array(sample_image)


      sample_image = tf.expand_dims(sample_image, axis=0)

      predictions = model.predict(sample_image)

      predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
      predicted_class = classes[predicted_class_index]

      return predicted_class

<hr>

##### IOT DATA

In [ ]:
def iot_data(pipeline,model,AQI,PM25,	PM10,	O3,	CO,	SO2,	NO2):
    X=np.array([AQI,PM25,PM10,O3,CO,SO2,NO2])
    X=X.reshape(1,-1)
    X=pipeline.transform(X)

    aqi_dict = { 0:'Good' ,
             1:'Moderate'  ,
             2:'Unhealthy_for_Sensitive_Groups' ,
             3:'Unhealthy',
             4:'Very_Unhealthy'  ,
             5:'Severe' }
    return aqi_dict[np.argmax(model.predict(X.reshape(1,-1)))]



<hr>

## GENERLIZE FUNCTION 

In [ ]:
def generelizePredict(model,satelite_model,iot_pipeline,iot_model,img_path,AQI=None,PM25=None,	PM10=None,	O3=None,	CO=None,	SO2=None,	NO2=None):
    preds=predict_deforestation_pollution(model, img_path)

    deforestation_prob = preds[0][0]
    pollution_prob = preds[0][1]

    classified=satelite_image_classification(satelite_model,img_path,img_size=(72,128))

    iot_predict=iot_data(iot_pipeline,iot_model,AQI,PM25,	PM10,	O3,	CO,	SO2,	NO2)

    prompt_str=f""" Deforestattion Probability is :{deforestation_prob} Pollution Probability is :{pollution_prob} Classified as :{classified} Air Quality: {iot_predict} """

    return prompt_str


<hr>

#### DEMO USEAGE

In [ ]:
generelizePredict(model,satelite_model,iot_pipeline,iot_model,"satellite-image-classification/data/green_area/Forest_1404.jpg",162,	193.00,	466.00	,225.00,	39.00,	8.00,	61.00)

### END
<hr>